## **Building a Memory-Enhanced Conversational Agent**






## Overview
This tutorial outlines the process of creating a conversational AI agent with enhanced memory capabilities. The agent incorporates both short-term and long-term memory to maintain context and improve the quality of interactions over time.

## Motivation  
Traditional chatbots often struggle with maintaining context beyond immediate interactions. This limitation can lead to disjointed conversations and a lack of personalization. By implementing both short-term and long-term memory, we aim to create an agent that can:

- Maintain context within a single conversation
- Remember important information across multiple sessions
-  Provide more coherent and personalized responses

## Key Components
1.  Language Model: The core AI component for understanding and generating responses.
2.  Short-term Memory: Stores the immediate conversation history.
3.  Long-term Memory: Retains important information across multiple conversations.
4.  Prompt Template: Structures the input for the language model, incorporating both types of memory.
5.  Memory Manager: Handles the storage and retrieval of information in both memory types.

# Method Details
## Setting Up the Environment
   1. Import necessary libraries for the language model, memory management, and prompt handling.
   2. Initialize the language model with desired parameters (e.g., model type, token limit).

## Implementing Memory Systems
 1. Create a store for short-term memory (conversation history):

    -  Use a dictionary to manage multiple conversation sessions.
    -  Implement a function to retrieve or create new conversation histories.
 2. Develop a long-term memory system:

    - Create a separate store for persistent information.
    - Implement functions to update and retrieve long-term memories.
    - Define simple criteria for what information to store long-term (e.g., longer user inputs).

## Designing the Conversation Structure
  1. Create a prompt template that includes:
    - A system message defining the AI's role.
    - A section for long-term memory context.
    - A placeholder for the conversation history (short-term memory).
The current user input.

## Building the Conversational Chain

   1. Combine the prompt template with the language model.
   2. Wrap this combination with a component that manages message history.
   3.  Ensure the chain can access and update both short-term and long-term memory.

## Creating the Interaction Loop
1. Develop a main chat function that:
   - Retrieves relevant long-term memories.
   - Invokes the conversational chain with the current input and memories.
   - Updates the long-term memory based on the interaction.
   - Returns the AI's response.

## Testing and Refinement
 1. Run example conversations to test the agent's ability to maintain context.
 2. Review both short-term and long-term memories after interactions.
 3. Adjust memory management criteria and prompt structure as needed.

In [ ]:
!pip install langchain langgraph  langchain_community langchain_core

In [21]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate , MessagesPlaceholder
from dotenv import load_dotenv
import os
import google.generativeai as gai
from google.colab import userdata
gai.configure(api_key = userdata.get('gemini_api'))
llm =   gai.GenerativeModel("gemini-pro")


## **Memory Stores**  

We will create both short-term( chat history )  and long term memory.


In [22]:
chat_store = {}
long_term_memory = {}
def get_chat_history(session_id) :
   if session_id not in chat_store :
     chat_store[session_id] = ChatMessageHistory()
   return chat_store[session_id]

def update_long_term_memory(session_id : str , input: str , output : str) :
  if session_id not in long_term_memory :
      long_term_memory[session_id] = []
  if len(input) :
    long_term_memory[session_id].append(f" User said: {input} ")
  if len(long_term_memory[session_id]) > 5 :
      long_term_memory[session_id][-5:]   # last  5 elements


def get_long_term_memory(session_id : str ) :
    return " , ".join(long_term_memory.get(session_id , []) )

## Prompt Template

In [33]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the information from long-term memory if relevant."),
    ("system", "Long-term memory: {long_term_memory}"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# Conversational Chain

In [ ]:
!pip install langchain openai
from langchain.chat_models import ChatOpenAI
os.environ["OPENAI_API_KEY"] = userdata.get('open_ai_api')
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

In [42]:
chain =  prompt  |  llm
chain_with_history = RunnableWithMessageHistory(
    chain ,
    get_chat_history ,
    input_messages_key="input",
    history_messages_key="history"
)

## Chat Function

In [49]:

def chat(input_text : str  , session_id : str ) :
  long_term_mem =  get_long_term_memory(session_id)
  response = chain_with_history.invoke({
      "input" :  input_text , "long_term_memory" :long_term_mem
  }, config={"configurable": {"session_id": session_id}})
  update_long_term_memory(session_id, input_text, response.content)
  return response.content

In [ ]:
session_id = "user_123"

print("AI:", chat("Hello! My name is Alice.", session_id))
print("AI:", chat("What's the weather like today?", session_id))
print("AI:", chat("I love sunny days.", session_id))
print("AI:", chat("Do you remember my name?", session_id))

## Review Memory


In [51]:
print("Conversation History:")
for message in chat_store[session_id].messages:
    print(f"{message.type}: {message.content}")

print("\nLong-term Memory:")
print(get_long_term_memory(session_id))

Conversation History:

Long-term Memory:

